In [1]:
tabular_data = {}
chroma_client = None

In [2]:
import os
import re
import pandas as pd
import chromadb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from typing import Any, List, Union
import langchain
from langchain import OpenAI, LLMChain
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.tools import DuckDuckGoSearchRun
from langchain.schema import AgentAction, AgentFinish
from langchain.memory import ConversationBufferWindowMemory
from pydantic import Field

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Set the environment variable for OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-2LEKhOKkL9aBQ5sc4RAbT3BlbkFJAY7GRuKKhijfw0XXuJTA"  # Replace with your actual API key



def load_excel(file):
    global tabular_data
    excel_data = pd.read_excel(file, sheet_name=None)
    text_data = {}
    for sheet_name, ddf in excel_data.items():
        tabular_data[sheet_name] = ddf
        tabular_data[sheet_name].columns = [str(_) for _ in ddf.columns]
        text_data[sheet_name] = ddf.select_dtypes(include=['object'])
    return text_data

def setup_chroma(text_data):
    global chroma_client
    chroma_client = chromadb.Client()
    for sheet_name, ddf in text_data.items():
        collection = chroma_client.create_collection(sheet_name.replace(" ","_"))
        for idx, row in ddf.iterrows():
            text = " ".join(str(val) for val in row if pd.notna(val))
            collection.add(documents=[text], metadatas=[{"row": idx}], ids=[f"{sheet_name}_{idx}"])

def get_data_overview():
    overview = "Data Overview:\n\n"
    for sheet_name, ddf in tabular_data.items():
        overview += f"Sheet Name of the following dataframe is from: {sheet_name}\n"
        overview += f"To Access Dataframe Code -- tabular_data['{sheet_name}']\n"
        overview += f"Shape of tabular_data['{sheet_name}']: {ddf.shape}\n"
        overview += f"Columns of tabular_data['{sheet_name}'] : {', '.join([str(_) for _ in ddf.columns])}\n"
        overview += f"Data Types of tabular_data['{sheet_name}']:\n{ddf.dtypes}\n"
        overview += f"First 2 rows of tabular_data['{sheet_name}']:\n{ddf.head(2).to_string()}\n\n"
    return overview

def execute_python_code(code: str) -> Any:
    global tabular_data
    
    if not tabular_data:
        return "No data has been loaded. Please load an Excel file first."
    if len(tabular_data) == 1:
        ddf = next(iter(tabular_data.values()))
    else:
        try:
            ddf = pd.concat(tabular_data.values(), keys=tabular_data.keys())
        except ValueError:
            return "Unable to concatenate data from multiple sheets. Please specify a sheet name."
    locals_dict = {
        'ddf': ddf, 'pd': pd, 'np': np, 
        'plt': plt, 'sns': sns, 
        'px': px, 'go': go
    }
    
    try:
        result = eval(code, globals(), locals_dict)
        
        if isinstance(result, (plt.Figure, go.Figure)):
            result.show()
            return "Graph Showed On UI"
        elif isinstance(result, pd.DataFrame):
            return result
        elif isinstance(result, pd.Series):
            return result
        elif isinstance(result, np.ndarray):
            return result.tolist()
        else:
            return result
    except Exception as e:
        return f"Error executing code: {str(e)}"



python_tool = Tool(
    name="PythonTool",
    func=execute_python_code,
    description="""
    Execute Python code for data insights and visualization.
    For accessing DataFrames use dataframe informations available above.
    Available libraries: pandas (pd), numpy (np), matplotlib.pyplot (plt), seaborn (sns), plotly.express (px), plotly.graph_objects (go).
    Ensure the action input is python code
    Examples:
    -"df['Column'].mean()"
    """
)

def query_chroma(query: str, sheet_name: str = None) -> str:
    global chroma_client
    
    try:
        if sheet_name:
            collection = chroma_client.get_collection(sheet_name.replace(" ", "_"))
            results = collection.query(query_texts=[query], n_results=5)
            return "\n".join(results['documents'][0])
        else:
            collections = chroma_client.list_collections()
            results = []
            for collection in collections:
                collection_obj = chroma_client.get_collection(collection.name)
                query_results = collection_obj.query(query_texts=[query], n_results=2)
                if query_results['documents']:
                    results.extend(query_results['documents'][0])
            return "\n".join(results)
    except Exception as e:
        return f"Failed to query Chroma DB: {e}"


chroma_tool = Tool(
    name="ChromaDBTool",
    func=query_chroma,
    description="""
    The data is also stored in Chroma DB if the question is not related to analytical you can use ChromaDBTool
    Query text data stored in Chroma DB. 
    Optionally provide a sheet name to search in a specific sheet.
    Examples:
    - 'find information about project deadlines'
    - 'Sheet3', 'find customer feedback'
    """
)

tools = [python_tool, chroma_tool]

class CustomPromptTemplate(StringPromptTemplate):
    template: str = Field()
    tools: List[Tool] = Field()

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.get("intermediate_steps", [])
        history = kwargs.get("chat_history", [])
        
        history_str = ""
        for entry in history:
            if isinstance(entry, tuple) and len(entry) == 2:
                human, ai = entry
                history_str += f"Human: {human}\nAI: {ai}\n\n"
            elif isinstance(entry, dict):
                history_str += f"Human: {entry.get('human', '')}\nAI: {entry.get('ai', '')}\n\n"

  

        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += f"Action: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\nThought: "
        
        kwargs["agent_scratchpad"] = thoughts
        kwargs["history"] = history_str
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        kwargs["get_data_overview"] = get_data_overview()
        return self.template.format(**kwargs)

prompt_template = CustomPromptTemplate(
    template="""
Answer the following questions as best you can. You have access to the following tools and dataframes:
Tools:
{tools}

Dataframes : 
{get_data_overview}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to be compassionate in your responses and consider the conversation history.
And try to use ChromaDB first to answer a question then go to Python tool.

Conversation history:
{history}

New question: {input}
{agent_scratchpad}""",
    input_variables=["input", "chat_history", "intermediate_steps"],
    tools=tools
)
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,            )
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

def load_and_process_excel(file_path):
    global tabular_data
    text_data = load_excel(file_path)
    setup_chroma(text_data)
    print(get_data_overview())

def main():
    # Load the Excel file
    excel_file_path = "Citywide_Payroll_Data__Fiscal_Year__-_Data_Dictionary.xlsx"  # Replace with your file path
    load_and_process_excel(excel_file_path)

    # Initialize the agent and tools
    output_parser = CustomOutputParser()
    
    llm = OpenAI(temperature=0)
    llm_chain = LLMChain(llm=llm, prompt=prompt_template)

    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=[tool.name for tool in tools]
    )

    memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history", return_messages=True)

    return AgentExecutor.from_agent_and_tools(
        agent=agent, 
        tools=tools, 
        verbose=True,
        memory=memory,
        max_iterations=5
    )




In [3]:
excel_file_path = "Citywide_Payroll_Data__Fiscal_Year__-_Data_Dictionary.xlsx"  # Replace with your file path
load_and_process_excel(excel_file_path)

# Initialize the agent and tools


D:\langchain_intros\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Data Overview:

Sheet Name of the following dataframe is from: Dataset Information
To Access Dataframe Code -- tabular_data['Dataset Information']
Shape of tabular_data['Dataset Information']: (20, 2)
Columns of tabular_data['Dataset Information'] : Unnamed: 0, Unnamed: 1
Data Types of tabular_data['Dataset Information']:
Unnamed: 0    object
Unnamed: 1    object
dtype: object
First 2 rows of tabular_data['Dataset Information']:
  Unnamed: 0 Unnamed: 1
0        NaN        NaN
1        NaN        NaN

Sheet Name of the following dataframe is from: Column Information
To Access Dataframe Code -- tabular_data['Column Information']
Shape of tabular_data['Column Information']: (17, 5)
Columns of tabular_data['Column Information'] : Data Dictionary - Column Information, Unnamed: 1, Unnamed: 2, Unnamed: 3, Unnamed: 4
Data Types of tabular_data['Column Information']:
Data Dictionary - Column Information    object
Unnamed: 1                              object
Unnamed: 2                         

In [4]:
output_parser = CustomOutputParser()

llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=[tool.name for tool in tools]
)

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history", return_messages=True)

agent = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True,
    memory=memory,
    max_iterations=5
)

D:\langchain_intros\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
D:\langchain_intros\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
D:\langchain_intros\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMSingleActionAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_a

In [5]:
langchain.debug = True

In [6]:
print(agent.invoke("What is the name of this DataSet?"))

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What is the name of this DataSet?",
  "chat_history": []
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "intermediate_steps": [],
  "stop": [
    "\nObservation:"
  ],
  "input": "What is the name of this DataSet?",
  "chat_history": []
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools and dataframes:\nTools:\nPythonTool: \n    Execute Python code for data insights and visualization.\n    For accessing DataFrames use dataframe informations available above.\n    Available libraries: pandas (pd), numpy (np), matplotlib.pyplot (plt), seaborn (sns), plotly.express (px), plotly.graph_objects (go).\n    Ensure the action input is python code\n    Examples:\n    -\"df['Column'].mean()\"\n    \nChromaDBTool: \n    The da

In [7]:
print(agent.invoke("Give me a pie graph to show the workers location and number of people working there?"))

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools and dataframes:\nTools:\nPythonTool: \n    Execute Python code for data insights and visualization.\n    For accessing DataFrames use dataframe informations available above.\n    Available libraries: pandas (pd), numpy (np), matplotlib.pyplot (plt), seaborn (sns), plotly.express (px), plotly.graph_objects (go).\n    Ensure the action input is python code\n    Examples:\n    -\"df['Column'].mean()\"\n    \nChromaDBTool: \n    The data is also stored in Chroma DB if the question is not related to analytical you can use ChromaDBTool\n    Query text data stored in Chroma DB. \n    Optionally provide a sheet name to search in 

In [9]:
answer = agent.invoke("""Any due dates in the data?""")
print(answer["output"])

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Answer the following questions as best you can. You have access to the following tools and dataframes:\nTools:\nPythonTool: \n    Execute Python code for data insights and visualization.\n    For accessing DataFrames use dataframe informations available above.\n    Available libraries: pandas (pd), numpy (np), matplotlib.pyplot (plt), seaborn (sns), plotly.express (px), plotly.graph_objects (go).\n    Ensure the action input is python code\n    Examples:\n    -\"df['Column'].mean()\"\n    \nChromaDBTool: \n    The data is also stored in Chroma DB if the question is not related to analytical you can use ChromaDBTool\n    Query text data stored in Chroma DB. \n    Optionally provide a sheet name to search in 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 25464 tokens (25208 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}